## LSTM

In [ ]:
# comment if not using google colab
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/My Drive/LSTM')

In [ ]:
# define seed
def seed_everything(seed=3407): # The torch.manual_seed(3407) is all you need! lol
    """
    Seed everything to make all operations in PyTorch (and other libraries) deterministic.
    Args:
        seed (int): Seed value to set.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import random
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

log_dir = './Result/untitled5/round13/'

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
tic = time.time()

seed_everything()

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Currently Using device: {device}')

acc_list = []

for z in range(5):
  print('='*20)
  print(f'Training of Fold {z+1}')
  print('='*20)

  filename = f'balanced_LSTM_long{z+1}.csv'
  print(f'Filename: {filename}')
  data = pd.read_csv(filename)
  
  feature_columns = [col for col in data.columns if col not in ('patientunitstayid', 'observationoffset', 'actualicumortality')]
  label_column = 'actualicumortality'

  # initialize lists
  sequences = []
  labels = []

  for _, group in data.groupby('patientunitstayid'):
    sequence = torch.tensor(group[feature_columns].values, dtype=torch.float)
    sequences.append(sequence)
    labels.append(group[label_column].iloc[0])

  # pad sequences & convert to tensor
  padded_sequences = pad_sequence(sequences, batch_first=True).to(device)
  # print(padded_sequences.shape)
  labels = torch.tensor(labels, dtype=torch.float).to(device)
  lengths = torch.tensor([len(seq) for seq in sequences]).to(device)

  # train-test split
  X_train, X_test, L_train, L_test, y_train, y_test = train_test_split(padded_sequences, lengths, labels, test_size=0.2, random_state=42)

  # minus 1 in time length
  L_test -= 1
  L_train -= 1
  
  # create dataloaders
  train_dataset = TensorDataset(X_train, L_train, y_train)
  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
  test_dataset = TensorDataset(X_test, L_test, y_test)
  test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

  # define LSTM model
  class TraditionalLSTM(nn.Module):
      def __init__(self, input_size, hidden_size, output_size):
        super(TraditionalLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

      def forward(self, input_seq, input_lengths):
        # Pack padded sequence
        packed_input = pack_padded_sequence(input_seq, input_lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (h, c) = self.lstm(packed_input)

        # Unpack output
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        idx = (input_lengths - 1).view(-1, 1).expand(len(input_lengths), output.size(2)).unsqueeze(1).to(device)
        last_output = output.gather(1, idx).squeeze(1)

        final_output = self.fc(last_output)
        final_output = self.sigmoid(final_output)
        return final_output

  # initializ
  input_size = 13
  hidden_size = 100
  output_size = 1

  model = TraditionalLSTM(input_size, hidden_size, output_size).to(device)
  criterion = nn.BCELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.002)

  # Train
  num_epochs = 40
  for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for sequences, lengths, labels in train_loader:
        sequences, lengths, labels = sequences.to(device), lengths.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences, lengths)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)

    # Test
    if epoch % 5 == 4:
      model.eval()
      predicted_labels_eval = []
      true_labels_eval = []
      with torch.no_grad():
        for sequences, lengths, labels in test_loader:
            sequences, lengths, labels = sequences.to(device), lengths.to(device), labels.to(device)
            outputs = model(sequences, lengths)
            predicted_labels_eval.extend(outputs.cpu().numpy().flatten())
            true_labels_eval.extend(labels.cpu().numpy().flatten())
      predicted_labels_eval = [1 if x >= 0.5 else 0 for x in predicted_labels_eval]
      accuracy = accuracy_score(true_labels_eval, predicted_labels_eval) * 100
      print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    else:
      print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}')

  # Eval
  model.eval()
  predicted_labels = []
  true_labels = []
  predicted_probs = []
  with torch.no_grad():
    for sequences, lengths, labels in test_loader:
        sequences, lengths, labels = sequences.to(device), lengths.to(device), labels.to(device)
        outputs = model(sequences, lengths)
        predicted_probs.extend(outputs.cpu().numpy().flatten())
        predicted_labels.extend(outputs.cpu().numpy().flatten())
        true_labels.extend(labels.cpu().numpy())

  # save predicted_labels
  results_path = os.path.join(log_dir, f'predicted_labels_fold{z+1}.npy')
  np.save(results_path, predicted_labels)
  # save true_labels
  results_path = os.path.join(log_dir, f'true_labels_fold{z+1}.npy')
  np.save(results_path, true_labels)
  # save model
  results_path = os.path.join(log_dir, f'model_fold{z+1}.pth')
  torch.save(model.state_dict(), results_path)

  from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
  predicted_labels = [1 if x >= 0.5 else 0 for x in predicted_labels]
  accuracy = accuracy_score(true_labels, predicted_labels)
  print(f'Accuracy: {accuracy}')
  acc_list.append(accuracy)

  precision = precision_score(true_labels, predicted_labels)
  print(f'Precision: {precision}')

  recall = recall_score(true_labels, predicted_labels)
  print(f'Recall: {recall}')

  f1 = f1_score(true_labels, predicted_labels)
  print(f'F1 Score: {f1}')

  conf_matrix = confusion_matrix(true_labels, predicted_labels)
  print(f'Confusion Matrix:\n{conf_matrix}')

  # ROC
  fpr, tpr, thresholds = roc_curve(true_labels, predicted_probs)
  roc_auc = auc(fpr, tpr)

  plt.figure()
  plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic')
  plt.legend(loc="lower right")
  plt.savefig(os.path.join(log_dir, f'roc_curve_fold{z+1}.png'))
  plt.show()

print(f'Accuracy: {acc_list}')
print(f'avg_Acc:{np.mean(acc_list)}')

toc = time.time()
print(f'Time taken: {toc - tic:0.4f} seconds')

### ROC for all

In [ ]:
predicted_labels = []
true_labels = []

for z in range(5):
    predicted_path = os.path.join(log_dir, f'predicted_labels_fold{z+1}.npy')
    true_path = os.path.join(log_dir, f'true_labels_fold{z+1}.npy')
    predicted_labels.append(np.load(predicted_path))
    true_labels.append(np.load(true_path))
predicted_labels = np.concatenate(predicted_labels)
true_labels = np.concatenate(true_labels)
fpr, tpr, thresholds = roc_curve(true_labels, predicted_labels)
roc_auc = auc(fpr, tpr)
plt.figure()

# # Optimal Threshold
# closest_zero_one_point = np.argmin(np.sqrt((fpr - 0)**2 + (tpr - 1)**2))
# closest_fpr = fpr[closest_zero_one_point]
# closest_tpr = tpr[closest_zero_one_point]
# closest_threshold = thresholds[closest_zero_one_point]
# plt.scatter(closest_fpr, closest_tpr, marker='o', color='red', label='Optimal Threshold')

plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.savefig(os.path.join(log_dir, f'roc_curve_total.svg'))
plt.show()